In [26]:
import numpy as np # linear algebra
import os

from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense , Dropout , Lambda, Flatten, Convolution2D, MaxPooling2D, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from random import shuffle

In [44]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=False,
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)

train_generator = train_datagen.flow_from_directory(
    directory='data/train/',
    target_size=(150,150),
    color_mode="rgb",
    batch_size=3,
    class_mode="categorical",
    shuffle=True
)

Found 93 images belonging to 5 classes.


In [45]:
valid_datagen = ImageDataGenerator(rescale=1./255)  

valid_generator = valid_datagen.flow_from_directory(
    directory='data/val/',
    target_size=(150,150),
    color_mode="rgb",
    batch_size=3,
    class_mode="categorical",
    shuffle=True
)

Found 25 images belonging to 5 classes.


In [55]:
model = Sequential([
        Convolution2D(256,(3,3), activation='relu', input_shape=(150,150,3)),
        BatchNormalization(axis=2),
        Convolution2D(128,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=2),
        Convolution2D(128,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=2),
        Convolution2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=2),
        Convolution2D(64,(3,3), activation='relu'),
        BatchNormalization(axis=2),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=2),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(5, activation='softmax')
        ])

model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 148, 148, 256)     7168      
_________________________________________________________________
batch_normalization_29 (Batc (None, 148, 148, 256)     592       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 146, 146, 128)     295040    
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 73, 73, 128)       0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 73, 73, 128)       292       
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 71, 71, 128)       147584    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 35, 35, 128)       0         
__________

In [56]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20)

Epoch 1/20
31/31 [==============================] - 22s 708ms/step - loss: 1.9845 - acc: 0.3011 - val_loss: 8.4682 - val_acc: 0.2500
Epoch 2/20
31/31 [==============================] - 16s 518ms/step - loss: 2.0706 - acc: 0.3011 - val_loss: 2.9016 - val_acc: 0.3182
Epoch 3/20
31/31 [==============================] - 16s 515ms/step - loss: 1.8292 - acc: 0.3333 - val_loss: 4.9590 - val_acc: 0.2273
Epoch 4/20
31/31 [==============================] - 16s 519ms/step - loss: 1.8839 - acc: 0.3226 - val_loss: 4.4740 - val_acc: 0.3636
Epoch 5/20
31/31 [==============================] - 16s 516ms/step - loss: 1.7407 - acc: 0.3226 - val_loss: 6.0501 - val_acc: 0.2727
Epoch 6/20
31/31 [==============================] - 16s 520ms/step - loss: 1.7881 - acc: 0.3656 - val_loss: 5.5922 - val_acc: 0.0909
Epoch 7/20
31/31 [==============================] - 16s 515ms/step - loss: 1.6995 - acc: 0.3763 - val_loss: 3.6560 - val_acc: 0.3182
Epoch 8/20
31/31 [==============================] - 16s 518ms/step - 

In [52]:
from keras.preprocessing import image

# predicting images
img = image.load_img('ben-affleck.jpg', target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])

classes = model.predict(images, batch_size=3)
print(classes)

[[0. 1. 0. 0. 0.]]


In [53]:
model.save('model.h5')